# MMA830 Assignment 2
## Team Oilers

## Load Packages

In [513]:
#%load_ext lab_black
#%load_ext autotime
from IPython import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

pd.set_option("display.max_colwidth", 300)
pd.options.display.max_columns = None

time: 7.96 ms


## Load Data

In [514]:
Promo = pd.read_excel("SoftDrinkSales-Data_origin.xlsx", header=[0, 1])
Promo.columns = ["Promo_%s%s" % (a, "_%s" % b if b else "") for a, b in Promo.columns]

Price = pd.read_excel(
    "SoftDrinkSales-Data_origin.xlsx", header=[0, 1], sheet_name="Price"
)
Price.columns = ["Price_%s%s" % (a, "_%s" % b if b else "") for a, b in Price.columns]

Bottle = pd.read_excel(
    "SoftDrinkSales-Data_origin.xlsx", header=[0, 1], sheet_name="Bottle"
)
Bottle.columns = [
    "Bottle_%s%s" % (a, "_%s" % b if b else "") for a, b in Bottle.columns
]

SizePromo = pd.read_excel(
    "SoftDrinkSales-Data_origin.xlsx", header=[0, 1], sheet_name="SizePromo"
)
SizePromo.columns = [
    "SizePromo_%s%s" % (a, "_%s" % b if b else "") for a, b in SizePromo.columns
]

Sales = pd.read_excel(
    "SoftDrinkSales-Data_origin.xlsx", header=[1, 2], sheet_name="Sales"
).iloc[:, :13]
Sales.columns = ["Sales_%s%s" % (a, "_%s" % b if b else "") for a, b in Sales.columns]

Empirical_rates = pd.read_excel(
    "SoftDrinkSales-Data_origin.xlsx", header=[0, 1], sheet_name="Empirical Rates"
)
Empirical_rates.columns = [
    "Empirical_rates_%s%s" % (a, "_%s" % b if b else "")
    for a, b in Empirical_rates.columns
]

time: 1.59 s


## Clean data

In [361]:
df_list = [
    Promo,
    Price,
    Bottle,
    SizePromo,
    Sales,
    # Empirical_rates,
]

time: 9.76 ms


In [363]:
Promo.head()

,Promo_Unnamed: 0_level_0_#Days,Promo_Brand_Segment,Promo_GOLD RUSH_VITAMIN,Promo_GOLD RUSH_DIET,Promo_GOLD RUSH_SPECIAL,Promo_GOLD RUSH_REGULAR,Promo_PWR_VITAMIN,Promo_PWR_DIET,Promo_PWR_SPECIAL,Promo_PWR_REGULAR,Promo_STORE BRAND_DIET,Promo_STORE BRAND_SPECIAL,Promo_STORE BRAND_REGULAR
0,27,0.000000,0.5488,0.4375,0.4636,0.4458,0.1960,0.4347,0.4220,0.3863,0.9862,0.6266,0.5815
1,34,0.019165,0.3882,0.5224,0.5543,0.5431,0.6220,0.5204,0.5572,0.6111,0.2912,0.2420,0.2590
2,41,0.038330,0.6328,0.7431,0.6776,0.6523,0.2211,0.2255,0.3886,0.4058,0.0103,0.3241,0.2892
3,48,0.057495,0.0841,0.3808,0.4405,0.3342,0.0024,0.2102,0.1079,0.3474,0.2838,0.5565,0.4791
4,55,0.076660,0.2894,0.4746,0.4826,0.4046,0.4799,0.6786,0.6086,0.5892,0.9909,0.2938,0.4497


time: 20 ms


In [364]:
for i in df_list:
    i.iloc[:, 1] = (i.iloc[:, 0] - i.iloc[0, 0]) / 365.25

time: 15.3 ms


In [365]:
new_df_list = []
for i in range(11):
    df = df_list[0].iloc[:, 1]
    for j in df_list:
        df = pd.concat([df, j.iloc[:, i + 2]], axis=1)
    new_df_list.append(df)

time: 131 ms


In [369]:
clean_df_list_train = []
clean_df_list_whole = []
for i in new_df_list:
    i.columns = [
        "Trend",
        "Promo",
        "Price",
        "Bottle",
        "SizePromo",
        "y",
        # "Products",
    ]
    clean_df_list_train.append(i.iloc[:148, :])
    clean_df_list_whole.append(i)

for j in clean_df_list_train:
    for i in range(1, 27):
        j["cos" + str(i)] = np.cos(2 * np.pi * j.iloc[:, 0] * i)
        j["sin" + str(i)] = np.cos(2 * np.pi * j.iloc[:, 0] * i)
for j in clean_df_list_whole:
    for i in range(1, 27):
        j["cos" + str(i)] = np.cos(2 * np.pi * j.iloc[:, 0] * i)
        j["sin" + str(i)] = np.cos(2 * np.pi * j.iloc[:, 0] * i)

time: 2.07 s


In [370]:
clean_df_list_train[0].head()

,Trend,Promo,Price,Bottle,SizePromo,y,cos1,sin1,cos2,sin2,cos3,sin3,cos4,sin4,cos5,sin5,cos6,sin6,cos7,sin7,cos8,sin8,cos9,sin9,cos10,sin10,cos11,sin11,cos12,sin12,cos13,sin13,cos14,sin14,cos15,sin15,cos16,sin16,cos17,sin17,cos18,sin18,cos19,sin19,cos20,sin20,cos21,sin21,cos22,sin22,cos23,sin23,cos24,sin24,cos25,sin25,cos26,sin26
0,0.000000,0.5488,1.7972,0,0,16355,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.019165,0.3882,1.7640,0,0,19224,0.992759,0.992759,0.971139,0.971139,0.935455,0.935455,0.886224,0.886224,0.824157,0.824157,0.750154,0.750154,0.665287,0.665287,0.570784,0.570784,0.468015,0.468015,0.358468,0.358468,0.243730,0.243730,0.125461,0.125461,0.005376,0.005376,-0.114788,-0.114788,-0.233289,-0.233289,-0.348411,-0.348411,-0.458487,-0.458487,-0.561923,-0.561923,-0.657221,-0.657221,-0.743001,-0.743001,-0.818020,-0.818020,-0.881192,-0.881192,-0.931601,-0.931601,-0.968519,-0.968519,-0.991410,-0.991410,-0.999942,-0.999942
2,0.038330,0.6328,1.6176,0,0,23265,0.971139,0.971139,0.886224,0.886224,0.750154,0.750154,0.570784,0.570784,0.358468,0.358468,0.125461,0.125461,-0.114788,-0.114788,-0.348411,-0.348411,-0.561923,-0.561923,-0.743001,-0.743001,-0.881192,-0.881192,-0.968519,-0.968519,-0.999942,-0.999942,-0.973648,-0.973648,-0.891153,-0.891153,-0.757220,-0.757220,-0.579579,-0.579579,-0.368484,-0.368484,-0.136120,-0.136120,0.104101,0.104101,0.338313,0.338313,0.552997,0.552997,0.735762,0.735762,0.876058,0.876058,0.965787,0.965787,0.999769,0.999769
3,0.057495,0.0841,1.9207,0,0,12253,0.935455,0.935455,0.750154,0.750154,0.468015,0.468015,0.125461,0.125461,-0.233289,-0.233289,-0.561923,-0.561923,-0.818020,-0.818020,-0.968519,-0.968519,-0.993993,-0.993993,-0.891153,-0.891153,-0.673275,-0.673275,-0.368484,-0.368484,-0.016127,-0.016127,0.338313,0.338313,0.649080,0.649080,0.876058,0.876058,0.989946,0.989946,0.976043,0.976043,0.836144,0.836144,0.588307,0.588307,0.264526,0.264526,-0.093402,-0.093402,-0.439273,-0.439273,-0.728438,-0.728438,-0.923571,-0.923571,-0.999480,-0.999480
4,0.076660,0.2894,1.8301,0,0,13452,0.886224,0.886224,0.570784,0.570784,0.125461,0.125461,-0.348411,-0.348411,-0.743001,-0.743001,-0.968519,-0.968519,-0.973648,-0.973648,-0.757220,-0.757220,-0.368484,-0.368484,0.104101,0.104101,0.552997,0.552997,0.876058,0.876058,0.999769,0.999769,0.895979,0.895979,0.588307,0.588307,0.146764,0.146764,-0.328176,-0.328176,-0.728438,-0.728438,-0.962943,-0.962943,-0.978326,-0.978326,-0.771089,-0.771089,-0.388388,-0.388388,0.082692,0.082692,0.534955,0.534955,0.865487,0.865487,0.999075,0.999075


time: 110 ms


## Build models

### Predict the rates

In [595]:
formula_part = ""
for i in list(clean_df_list_train[0].columns):
    formula_part = formula_part + " + " + i

formula = "np.log(y) ~ " + formula_part
Pridicted_Rates = pd.DataFrame()

eleven_models = []
for i in range(len(clean_df_list_train)):
    results = ols(formula, clean_df_list_train[i]).fit()
    eleven_models.append(results)
    pred = np.exp(results.predict(clean_df_list_whole[i]))
    Pridicted_Rates = pd.concat([Pridicted_Rates, pd.DataFrame(pred)], axis=1)

Pridicted_Rates["sum"] = Pridicted_Rates.sum(axis=1)
for i in range(11):
    Pridicted_Rates.iloc[:, i] = Pridicted_Rates.iloc[:, i] / Pridicted_Rates["sum"]

Sales_forecast = Pridicted_Rates.iloc[:, -1]
Sales_forecast.to_excel("Sales_forest.xlsx")
Pridicted_Rates = Pridicted_Rates.iloc[:, :-1]
Pridicted_Rates.to_excel("Pridicted_Rates.xlsx")

Pridicted_Rates
print(
    "RMSE of Predicted Rates - Calibration: ",
    sqrt(MSE(Empirical_rates.iloc[:148, 2:], Pridicted_Rates[:148])),
)
print(
    "RMSE of Predicted Rates - Holdout: ",
    sqrt(MSE(Empirical_rates.iloc[148:, 2:], Pridicted_Rates[148:])),
)

,0,0,0,0,0,0,0,0,0,0,0
0,0.021072,0.064043,0.044416,0.091059,0.040919,0.076350,0.200774,0.258090,0.007978,0.078308,0.116991
1,0.023298,0.061906,0.046900,0.090869,0.056156,0.080817,0.224177,0.284714,0.006196,0.049299,0.075667
2,0.030359,0.082886,0.056986,0.117720,0.041671,0.068444,0.199634,0.241906,0.005342,0.059728,0.095324
3,0.018575,0.067189,0.048763,0.097075,0.048137,0.075852,0.201007,0.252000,0.007271,0.076073,0.108057
4,0.016340,0.057461,0.044031,0.083720,0.042927,0.092074,0.216649,0.279461,0.010321,0.059060,0.097955
...,...,...,...,...,...,...,...,...,...,...,...
155,0.014013,0.095228,0.062234,0.107953,0.042399,0.103962,0.188743,0.245628,0.014431,0.052256,0.073152
156,0.015287,0.097889,0.062141,0.107305,0.041787,0.103111,0.185517,0.245118,0.014563,0.053969,0.073313
157,0.015110,0.097207,0.061595,0.106823,0.041722,0.104638,0.184664,0.247170,0.014999,0.053227,0.072845
158,0.014677,0.097682,0.063060,0.108180,0.041608,0.107188,0.183579,0.246894,0.014359,0.051068,0.071704


RMSE of Predicted Rates - Calibration:  0.0026647814471301473
RMSE of Predicted Rates - Holdout:  0.06749615774187415
time: 3.64 s


## Predict the sales

In [597]:
Actual_Sales = pd.read_excel(
    "SoftDrinkSales-Data_origin.xlsx", header=[1, 2], sheet_name="Sales"
).iloc[:148, 13]

pd.DataFrame(Sales_forecast[:148])
print(
    "SUM (Residual Square): ", (np.sum(np.square(Sales_forecast[:148] - Actual_Sales)))
)

,sum
0,7.667585e+05
1,8.220275e+05
2,7.859522e+05
3,6.846940e+05
4,7.968700e+05
...,...
143,1.004530e+06
144,1.035285e+06
145,1.258046e+06
146,1.008202e+06


SUM (Residual Square):  33589398324.160686
time: 275 ms


## Optimize Price

In [599]:
from scipy.optimize import minimize

time: 999 µs


In [600]:
day = 1112
temp = Promo.reset_index()
day_index = temp[temp.iloc[:, 1] == day].index

time: 6.98 ms


In [601]:
day_1112_df_list = []
for i in range(len(clean_df_list_whole)):
    day_1112_df_list.append(clean_df_list_whole[i].iloc[day_index, :])

time: 16 ms


In [581]:
def profit(price_list):
    sales_results = []
    for i in range(len(price_list)):
        day_1112_df_list[i].Price = price_list[i]
        sales_results.append(
            float(np.exp(eleven_models[i].predict(day_1112_df_list[i])))
        )
    sales_results = np.array(sales_results)

    return -np.sum((sales_results * (price_list - cost_GOLD_RUSH)))


def con(price_GOLD_RUSH):
    return np.sum(price_GOLD_RUSH > cost_GOLD_RUSH) - 4

time: 16 ms


In [571]:
price_GOLD_RUSH = np.array([2, 2, 3, 3])
cost_GOLD_RUSH = np.array([1.3, 1.2, 1.3, 1.2])
opt = minimize(
    profit, price_GOLD_RUSH, method="BFGS", constraints={"type": "eq", "fun": con}
)

time: 2min 8s


In [572]:
opt

      fun: -282310.056154544
 hess_inv: array([[ 7.21591722e-04, -2.87244080e-05,  1.81502691e-04,
        -2.70044507e-04],
       [-2.87244080e-05,  1.74420136e-04, -4.95596202e-05,
        -1.04176075e-05],
       [ 1.81502691e-04, -4.95596202e-05,  5.51270638e-04,
         1.84883978e-04],
       [-2.70044507e-04, -1.04176075e-05,  1.84883978e-04,
         2.27464255e-04]])
      jac: array([0.       , 0.       , 0.0078125, 0.       ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 223
      nit: 19
     njev: 37
   status: 2
  success: False
        x: array([ 4.58890118,  4.5464985 ,  6.22974096, 10.76474824])

time: 6.99 ms


In [591]:
opt.x

array([ 4.58890118,  4.5464985 ,  6.22974096, 10.76474824])

time: 3.99 ms


In [592]:
print("The best price is ", opt.x)

The best price is  [ 4.58890118  4.5464985   6.22974096 10.76474824]
time: 4.05 ms
